In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

from matplotlib import pyplot as plt

pd.set_option('display.max_rows', None)

In [89]:
#Open csv file.

data = pd.read_csv("/content/drive/MyDrive/NTDB-Epidural/combined_data.csv", index_col=0)
data.shape

(13408, 137)

#Initial Renaming and Merging Response Values

In [4]:
#See all columns.

print(list(data.columns))

['AGEYEARS', 'SEX', 'WHITE', 'ASIAN', 'BLACK', 'AMERICANINDIAN', 'PACIFICISLANDER', 'RACEOTHER', 'ETHNICITY', 'WEIGHT', 'HEIGHT', 'SBP', 'PULSERATE', 'SUPPLEMENTALOXYGEN', 'PULSEOXIMETRY', 'RESPIRATORYASSISTANCE', 'RESPIRATORYRATE', 'TEMPERATURE', 'PREHOSPITALCARDIACARREST', 'GCSEYE', 'GCSVERBAL', 'GCSMOTOR', 'TOTALGCS', 'TBIPUPILLARYRESPONSE', 'TBIMIDLINESHIFT', 'CC_SMOKING', 'CC_ALCOHOLISM', 'CC_SUBSTANCEABUSE', 'CC_DIABETES', 'CC_HYPERTENSION', 'CC_CHF', 'CC_MI', 'CC_ANGINAPECTORIS', 'CC_CVA', 'CC_PAD', 'CC_COPD', 'CC_RENAL', 'CC_CIRRHOSIS', 'CC_BLEEDING', 'CC_DISCANCER', 'CC_CHEMO', 'CC_DEMENTIA', 'CC_ADHD', 'CC_MENTALPERSONALITY', 'CC_FUNCTIONAL', 'CC_PREGNANCY', 'CC_ANTICOAGULANT', 'CC_STEROID', 'CC_ADLC', 'HOSPITALARRIVALDAYS', 'TRANSPORTMODE', 'INTERFACILITYTRANSFER', 'TRAUMATYPE', 'INTENT', 'MECHANISM', 'PROTDEV_AIRBAG_PRESENT', 'PROTDEV_CHILD_RESTRAINT', 'PROTDEV_EYE_PROTECT', 'PROTDEV_HELMET', 'PROTDEV_LAP_BELT', 'PROTDEV_NONE', 'PROTDEV_OTHER', 'PROTDEV_PER_FLOAT', 'PROTDEV

In [5]:
#Change response values to strings for 'ALCOHOLSCREEN'.

data.loc[data['ALCOHOLSCREEN'] == 1, 'ALCOHOLSCREEN'] = 'Yes'
data.loc[data['ALCOHOLSCREEN'] == 2, 'ALCOHOLSCREEN'] = 'No'

data['ALCOHOLSCREEN'].value_counts(normalize=False, dropna=False)

No     7946
Yes    5389
NaN      73
Name: ALCOHOLSCREEN, dtype: int64

In [6]:
#Assign '0' for patients who were not screened for alcohol.

data['ALCOHOLSCREENRESULT'] = data['ALCOHOLSCREENRESULT'].fillna(0)

In [7]:
#Merge race columns and drop former columns.

data.loc[data['AMERICANINDIAN'] == 1, 'RACE'] = 'American Indian'
data.loc[data['ASIAN'] == 1, 'RACE'] = 'Asian'
data.loc[data['BLACK'] == 1, 'RACE'] = 'Black'
data.loc[data['PACIFICISLANDER'] == 1, 'RACE'] = 'Pacific Islander'
data.loc[data['RACEOTHER'] == 1, 'RACE'] = 'Other'
data.loc[data['WHITE'] == 1, 'RACE'] = 'White'

data = data.drop(columns=['AMERICANINDIAN', 'ASIAN', 'BLACK', 'PACIFICISLANDER', 'RACEOTHER', 'WHITE'])

data['RACE'].value_counts(normalize=False, dropna=False)

White               9368
Other               1560
Black               1293
Asian                537
NaN                  416
American Indian      160
Pacific Islander      74
Name: RACE, dtype: int64

In [8]:
#Change response values to strings for 'ANTIBIOTICTHERAPY'.

data.loc[data['ANTIBIOTICTHERAPY'] == 1, 'ANTIBIOTICTHERAPY'] = 'Yes'
data.loc[data['ANTIBIOTICTHERAPY'] == 2, 'ANTIBIOTICTHERAPY'] = 'No'

data['ANTIBIOTICTHERAPY'].value_counts(normalize=False, dropna=False)

NaN    12655
Yes      594
No       159
Name: ANTIBIOTICTHERAPY, dtype: int64

In [9]:
#Change response values to strings for 'BEDSIZE'.

data.loc[data['BEDSIZE'] == 1, 'BEDSIZE'] = '200 or fewer'
data.loc[data['BEDSIZE'] == 2, 'BEDSIZE'] = '201 to 400'
data.loc[data['BEDSIZE'] == 3, 'BEDSIZE'] = '401 to 600'
data.loc[data['BEDSIZE'] == 4, 'BEDSIZE'] = 'More than 600'
data = data.dropna(subset=['BEDSIZE'])

data['BEDSIZE'].value_counts(normalize=False, dropna=False)

201 to 400       4200
More than 600    4041
401 to 600       3897
200 or fewer     1269
Name: BEDSIZE, dtype: int64

In [10]:
#Assign male patients' pregnancy response values as 'Not applicable (male patient)

data.loc[data['SEX'] == 1, 'CC_PREGNANCY'] = 'Not applicable (male patient)'

data['CC_PREGNANCY'].value_counts(normalize=False, dropna=False)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Not applicable (male patient)    9389
No                               3990
Unknown                            20
Yes                                 8
Name: CC_PREGNANCY, dtype: int64

In [11]:
#Change response values to strings for 'DRGSCR_AMPHETAMINE'.

data.loc[data['DRGSCR_AMPHETAMINE'] == 1, 'DRGSCR_AMPHETAMINE'] = 'Yes'
data.loc[data['DRGSCR_AMPHETAMINE'] == 0, 'DRGSCR_AMPHETAMINE'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_AMPHETAMINE'] = 'Not tested'
data['DRGSCR_AMPHETAMINE'] = data['DRGSCR_AMPHETAMINE'].fillna('Not tested')

data['DRGSCR_AMPHETAMINE'].value_counts(normalize=False, dropna=False)

<ipython-input-11-57176db88bb1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DRGSCR_AMPHETAMINE'] = data['DRGSCR_AMPHETAMINE'].fillna('Not tested')


Not tested    9172
No            3777
Yes            458
Name: DRGSCR_AMPHETAMINE, dtype: int64

In [12]:
#Change response values to strings for 'DRGSCR_BARBITURATE'.

data.loc[data['DRGSCR_BARBITURATE'] == 1, 'DRGSCR_BARBITURATE'] = 'Yes'
data.loc[data['DRGSCR_BARBITURATE'] == 0, 'DRGSCR_BARBITURATE'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_BARBITURATE'] = 'Not tested'
data['DRGSCR_BARBITURATE'] = data['DRGSCR_BARBITURATE'].fillna('Not tested')

data['DRGSCR_BARBITURATE'].value_counts(normalize=False, dropna=False)

<ipython-input-12-209bd535a19f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DRGSCR_BARBITURATE'] = data['DRGSCR_BARBITURATE'].fillna('Not tested')


Not tested    9172
No            4183
Yes             52
Name: DRGSCR_BARBITURATE, dtype: int64

In [13]:
#Change response values to strings for 'DRGSCR_BENZODIAZEPINES'.

data.loc[data['DRGSCR_BENZODIAZEPINES'] == 1, 'DRGSCR_BENZODIAZEPINES'] = 'Yes'
data.loc[data['DRGSCR_BENZODIAZEPINES'] == 0, 'DRGSCR_BENZODIAZEPINES'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_BENZODIAZEPINES'] = 'Not tested'
data['DRGSCR_BENZODIAZEPINES'] = data['DRGSCR_BENZODIAZEPINES'].fillna('Not tested')

data['DRGSCR_BENZODIAZEPINES'].value_counts(normalize=False, dropna=False)

<ipython-input-13-7a49e48b66fa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DRGSCR_BENZODIAZEPINES'] = data['DRGSCR_BENZODIAZEPINES'].fillna('Not tested')


Not tested    9172
No            3954
Yes            281
Name: DRGSCR_BENZODIAZEPINES, dtype: int64

In [14]:
#Change response values to strings for 'DRGSCR_CANNABINOID'.

data.loc[data['DRGSCR_CANNABINOID'] == 1, 'DRGSCR_CANNABINOID'] = 'Yes'
data.loc[data['DRGSCR_CANNABINOID'] == 0, 'DRGSCR_CANNABINOID'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_CANNABINOID'] = 'Not tested'
data['DRGSCR_CANNABINOID'] = data['DRGSCR_CANNABINOID'].fillna('Not tested')

data['DRGSCR_CANNABINOID'].value_counts(normalize=False, dropna=False)

<ipython-input-14-c62bd255599f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DRGSCR_CANNABINOID'] = data['DRGSCR_CANNABINOID'].fillna('Not tested')


Not tested    9172
No            3084
Yes           1151
Name: DRGSCR_CANNABINOID, dtype: int64

In [15]:
#Change response values to strings for 'DRGSCR_COCAINE'.

data.loc[data['DRGSCR_COCAINE'] == 1, 'DRGSCR_COCAINE'] = 'Yes'
data.loc[data['DRGSCR_COCAINE'] == 0, 'DRGSCR_COCAINE'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_COCAINE'] = 'Not tested'
data['DRGSCR_COCAINE'] = data['DRGSCR_COCAINE'].fillna('Not tested')

data['DRGSCR_COCAINE'].value_counts(normalize=False, dropna=False)

Not tested    9172
No            3935
Yes            300
Name: DRGSCR_COCAINE, dtype: int64

In [16]:
#Change response values to strings for 'DRGSCR_ECSTASY'.

data.loc[data['DRGSCR_ECSTASY'] == 1, 'DRGSCR_ECSTASY'] = 'Yes'
data.loc[data['DRGSCR_ECSTASY'] == 0, 'DRGSCR_ECSTASY'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_ECSTASY'] = 'Not tested'
data['DRGSCR_ECSTASY'] = data['DRGSCR_ECSTASY'].fillna('Not tested')

data['DRGSCR_ECSTASY'].value_counts(normalize=False, dropna=False)

Not tested    9172
No            4193
Yes             42
Name: DRGSCR_ECSTASY, dtype: int64

In [17]:
#Change response values to strings for 'DRGSCR_METHADONE'.

data.loc[data['DRGSCR_METHADONE'] == 1, 'DRGSCR_METHADONE'] = 'Yes'
data.loc[data['DRGSCR_METHADONE'] == 0, 'DRGSCR_METHADONE'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_METHADONE'] = 'Not tested'
data['DRGSCR_METHADONE'] = data['DRGSCR_METHADONE'].fillna('Not tested')

data['DRGSCR_METHADONE'].value_counts(normalize=False, dropna=False)

Not tested    9172
No            4200
Yes             35
Name: DRGSCR_METHADONE, dtype: int64

In [18]:
#Change response values to strings for 'DRGSCR_METHAMPHETAMINE'.

data.loc[data['DRGSCR_METHAMPHETAMINE'] == 1, 'DRGSCR_METHAMPHETAMINE'] = 'Yes'
data.loc[data['DRGSCR_METHAMPHETAMINE'] == 0, 'DRGSCR_METHAMPHETAMINE'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_METHAMPHETAMINE'] = 'Not tested'
data['DRGSCR_METHAMPHETAMINE'] = data['DRGSCR_METHAMPHETAMINE'].fillna('Not tested')

data['DRGSCR_METHAMPHETAMINE'].value_counts(normalize=False, dropna=False)

Not tested    9172
No            4096
Yes            139
Name: DRGSCR_METHAMPHETAMINE, dtype: int64

In [19]:
#Change response values to strings for 'DRGSCR_OPIOID'.

data.loc[data['DRGSCR_OPIOID'] == 1, 'DRGSCR_OPIOID'] = 'Yes'
data.loc[data['DRGSCR_OPIOID'] == 0, 'DRGSCR_OPIOID'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_OPIOID'] = 'Not tested'
data['DRGSCR_OPIOID'] = data['DRGSCR_OPIOID'].fillna('Not tested')

data['DRGSCR_OPIOID'].value_counts(normalize=False, dropna=False)

Not tested    9172
No            3988
Yes            247
Name: DRGSCR_OPIOID, dtype: int64

In [20]:
#Change response values to strings for 'DRGSCR_OXYCODONE'.

data.loc[data['DRGSCR_OXYCODONE'] == 1, 'DRGSCR_OXYCODONE'] = 'Yes'
data.loc[data['DRGSCR_OXYCODONE'] == 0, 'DRGSCR_OXYCODONE'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_OXYCODONE'] = 'Not tested'
data['DRGSCR_AMPHETAMINE'] = data['DRGSCR_AMPHETAMINE'].fillna('Not tested')

data['DRGSCR_OXYCODONE'].value_counts(normalize=False, dropna=False)

Not tested    9172
No            4203
Yes             32
Name: DRGSCR_OXYCODONE, dtype: int64

In [21]:
#Change response values to strings for 'DRGSCR_PHENCYCLIDINE'.

data.loc[data['DRGSCR_PHENCYCLIDINE'] == 1, 'DRGSCR_PHENCYCLIDINE'] = 'Yes'
data.loc[data['DRGSCR_PHENCYCLIDINE'] == 0, 'DRGSCR_PHENCYCLIDINE'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_PHENCYCLIDINE'] = 'Not tested'
data['DRGSCR_AMPHETAMINE'] = data['DRGSCR_AMPHETAMINE'].fillna('Not tested')

data['DRGSCR_PHENCYCLIDINE'].value_counts(normalize=False, dropna=False)

Not tested    9172
No            4220
Yes             15
Name: DRGSCR_PHENCYCLIDINE, dtype: int64

In [22]:
#Change response values to strings for 'DRGSCR_TRICYCLICDEPRESS'.

data.loc[data['DRGSCR_TRICYCLICDEPRESS'] == 1, 'DRGSCR_TRICYCLICDEPRESS'] = 'Yes'
data.loc[data['DRGSCR_TRICYCLICDEPRESS'] == 0, 'DRGSCR_TRICYCLICDEPRESS'] = 'No'
data.loc[data['DRGSCR_NOTTESTED'] == 1, 'DRGSCR_TRICYCLICDEPRESS'] = 'Not tested'
data['DRGSCR_AMPHETAMINE'] = data['DRGSCR_AMPHETAMINE'].fillna('Not tested')

data['DRGSCR_TRICYCLICDEPRESS'].value_counts(normalize=False, dropna=False)

Not tested    9172
No            4232
Yes              3
Name: DRGSCR_TRICYCLICDEPRESS, dtype: int64

In [23]:
#Drop 'DRGSCR_NOTTESTED' since it will not be utilized from this point.

data = data.drop(columns=['DRGSCR_NOTTESTED'])

In [24]:
#Change response values to strings for 'ETHNICITY'.

data.loc[data['ETHNICITY'] == 1, 'ETHNICITY'] = 'Hispanic or Latino'
data.loc[data['ETHNICITY'] == 2, 'ETHNICITY'] = 'Not Hispanic or Latino'

data['ETHNICITY'].value_counts(normalize=False, dropna=False)

Not Hispanic or Latino    10286
Hispanic or Latino         2569
NaN                         552
Name: ETHNICITY, dtype: int64

In [25]:
#Change response values to strings for 'HOSPDISCHARGEDISPOSITION'.

data.loc[data['HOSPDISCHARGEDISPOSITION'] == 1, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to a short-term general hospital for inpatient care'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 2, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to an Intermediate Care Facility (ICF)'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 3, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to home under care of organized home health service'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 4, 'HOSPDISCHARGEDISPOSITION'] = 'Left against medical advice or discontinued care'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 5, 'HOSPDISCHARGEDISPOSITION'] = 'Deceased/Expired'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 6, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged to home or self-care (routine discharge)'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 7, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to Skilled Nursing Facility (SNF)'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 8, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to hospice care'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 10, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to court/law enforcement.'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 11, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to inpatient rehab or designated unit'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 12, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to Long Term Care Hospital (LTCH)'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 13, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to a psychiatric hospital or psychiatric distinct part unit of a hospital'
data.loc[data['HOSPDISCHARGEDISPOSITION'] == 14, 'HOSPDISCHARGEDISPOSITION'] = 'Discharged/Transferred to another type of institution not defined elsewhere'

data['HOSPDISCHARGEDISPOSITION'].value_counts(normalize=False, dropna=False)

Discharged to home or self-care (routine discharge)                                                 9346
NaN                                                                                                 1121
Discharged/Transferred to inpatient rehab or designated unit                                         935
Discharged/Transferred to home under care of organized home health service                           515
Discharged/Transferred to Skilled Nursing Facility (SNF)                                             467
Deceased/Expired                                                                                     282
Left against medical advice or discontinued care                                                     231
Discharged/Transferred to a short-term general hospital for inpatient care                           183
Discharged/Transferred to Long Term Care Hospital (LTCH)                                              97
Discharged/Transferred to court/law enforcement.       

In [26]:
#Change response values to strings for 'HOSPITALTYPE'.

data.loc[data['HOSPITALTYPE'] == 1, 'HOSPITALTYPE'] = 'For Profit'
data.loc[data['HOSPITALTYPE'] == 2, 'HOSPITALTYPE'] = 'Non-profit'
data.loc[data['HOSPITALTYPE'] == 3, 'HOSPITALTYPE'] = 'Government'

data['HOSPITALTYPE'].value_counts(normalize=False, dropna=False)

Non-profit    12171
For Profit     1064
NaN             114
Government       58
Name: HOSPITALTYPE, dtype: int64

In [27]:
#Merge ICP columns and drop former columns.

data.loc[data['ICPEVDRAIN'] == 1, 'ICP'] = 'Intraventricular drain/catheter (e.g. ventriculostomy, external ventricular drain)'
data.loc[data['ICPJVBULB'] == 1, 'ICP'] = 'Jugular venous bulb'
data.loc[data['ICPO2MONITOR'] == 1, 'ICP'] = 'Intraparenchymal oxygen monitor (e.g. Licox)'
data.loc[data['ICPPARENCH'] == 1, 'ICP'] = 'Intraparenchymal pressure monitor (e.g. Camino bolt, subarachnoid bolt, intraparenchymal catheter)'
data.loc[data['ICPNONE'] == 1, 'ICP'] = 'None'

data = data.drop(columns=['ICPEVDRAIN', 'ICPJVBULB', 'ICPO2MONITOR', 'ICPPARENCH', 'ICPNONE'])

data['ICP'].value_counts(normalize=False, dropna=False)

None                                                                                                  11833
NaN                                                                                                    1054
Intraventricular drain/catheter (e.g. ventriculostomy, external ventricular drain)                      272
Intraparenchymal pressure monitor (e.g. Camino bolt, subarachnoid bolt, intraparenchymal catheter)      234
Jugular venous bulb                                                                                      11
Intraparenchymal oxygen monitor (e.g. Licox)                                                              3
Name: ICP, dtype: int64

In [28]:
#Change response values to strings for 'INTENT'.

data.loc[data['INTENT'] == 1, 'INTENT'] = 'Unintentional'
data.loc[data['INTENT'] == 2, 'INTENT'] = 'Self-inflicted'
data.loc[data['INTENT'] == 3, 'INTENT'] = 'Assault'
data.loc[data['INTENT'] == 4, 'INTENT'] = 'Undetermined'
data.loc[data['INTENT'] == 5, 'INTENT'] = 'Other'

data['INTENT'].value_counts(normalize=False, dropna=False)

Unintentional     11845
Assault            1292
NaN                 126
Self-inflicted       64
Undetermined         63
Other                17
Name: INTENT, dtype: int64

In [29]:
#Change response values to strings for 'INTERFACILITYTRANSFER'.

data.loc[data['INTERFACILITYTRANSFER'] == 1, 'INTERFACILITYTRANSFER'] = 'Yes'
data.loc[data['INTERFACILITYTRANSFER'] == 2, 'INTERFACILITYTRANSFER'] = 'No'

data['INTERFACILITYTRANSFER'].value_counts(normalize=False, dropna=False)

No     7763
Yes    5640
NaN       4
Name: INTERFACILITYTRANSFER, dtype: int64

In [30]:
#Change response values to strings for 'MECHANISM'.

data.loc[data['MECHANISM'] == 1, 'MECHANISM'] = 'Cut/Pierce'
data.loc[data['MECHANISM'] == 2, 'MECHANISM'] = 'Drowning/Submersion'
data.loc[data['MECHANISM'] == 3, 'MECHANISM'] = 'Fall'
data.loc[data['MECHANISM'] == 4, 'MECHANISM'] = 'Fire/Flame'
data.loc[data['MECHANISM'] == 5, 'MECHANISM'] = 'Hot Object/Substance'
data.loc[data['MECHANISM'] == 6, 'MECHANISM'] = 'Firearm'
data.loc[data['MECHANISM'] == 7, 'MECHANISM'] = 'Machinery'
data.loc[data['MECHANISM'] == 8, 'MECHANISM'] = 'MVT Occupant'
data.loc[data['MECHANISM'] == 9, 'MECHANISM'] = 'MVT Motorcyclist'
data.loc[data['MECHANISM'] == 10, 'MECHANISM'] = 'MVT Pedal Cyclist'
data.loc[data['MECHANISM'] == 11, 'MECHANISM'] = 'MVT Pedestrian'
data.loc[data['MECHANISM'] == 12, 'MECHANISM'] = 'MVT Unspecified'
data.loc[data['MECHANISM'] == 13, 'MECHANISM'] = 'MVT Other'
data.loc[data['MECHANISM'] == 14, 'MECHANISM'] = 'Pedal Cyclist, Other'
data.loc[data['MECHANISM'] == 15, 'MECHANISM'] = 'Pedestrian, Other'
data.loc[data['MECHANISM'] == 16, 'MECHANISM'] = 'Transport, Other'
data.loc[data['MECHANISM'] == 17, 'MECHANISM'] = 'Natural/Environmental, Bites and Stings'
data.loc[data['MECHANISM'] == 18, 'MECHANISM'] = 'Natural/Environmental, Other'
data.loc[data['MECHANISM'] == 19, 'MECHANISM'] = 'Overexertion'
data.loc[data['MECHANISM'] == 20, 'MECHANISM'] = 'Poisoning'
data.loc[data['MECHANISM'] == 21, 'MECHANISM'] = 'Struck By/Against'
data.loc[data['MECHANISM'] == 22, 'MECHANISM'] = 'Suffocation'
data.loc[data['MECHANISM'] == 23, 'MECHANISM'] = 'Other Specified and Classifiable'
data.loc[data['MECHANISM'] == 24, 'MECHANISM'] = 'Other Specified, Not Elsewhere Classifiable'
data.loc[data['MECHANISM'] == 25, 'MECHANISM'] = 'Unspecified'
data.loc[data['MECHANISM'] == 26, 'MECHANISM'] = 'Adverse Effects, Medical Care'
data.loc[data['MECHANISM'] == 27, 'MECHANISM'] = 'Adverse Effects, Drugs'
data.loc[data['MECHANISM'] == 30, 'MECHANISM'] = 'Pedestrian Pedal'
data.loc[data['MECHANISM'] == 31, 'MECHANISM'] = 'MVT Occupant and Others'
data.loc[data['MECHANISM'] == 32, 'MECHANISM'] = 'Others'

data['MECHANISM'].value_counts(normalize=False, dropna=False)

Fall                                           6682
MVT Occupant                                   1671
Struck By/Against                              1539
Transport, Other                                629
MVT Motorcyclist                                448
NaN                                             442
Pedal Cyclist, Other                            433
MVT Pedestrian                                  428
MVT Other                                       248
MVT Pedal Cyclist                               218
Firearm                                         108
Pedestrian, Other                               105
Other Specified and Classifiable                100
Natural/Environmental, Other                     90
Cut/Pierce                                       62
Other Specified, Not Elsewhere Classifiable      58
MVT Unspecified                                  58
Unspecified                                      53
Machinery                                        16
Overexertion

In [31]:
#Merge 'PATIENTSOCCUPATION' and 'WORKRELATED' and change response values to strings for.

data.loc[data['WORKRELATED'] == 2, 'WORKRELATED'] = 'No'
data.loc[data['WORKRELATED'] == 1, 'WORKRELATED'] = None
data.loc[data['PATIENTSOCCUPATION'] == 1, 'WORKRELATED'] = 'Yes (Business and Financial Operations Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 2, 'WORKRELATED'] = 'Yes (Architecture and Engineering Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 3, 'WORKRELATED'] = 'Yes (Community and Social Services Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 4, 'WORKRELATED'] = 'Yes (Education, Training, and Library Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 5, 'WORKRELATED'] = 'Yes (Healthcare Practitioners and Technical Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 6, 'WORKRELATED'] = 'Yes (Protective Service Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 7, 'WORKRELATED'] = 'Yes (Building and Grounds Cleaning and Maintenance)'
data.loc[data['PATIENTSOCCUPATION'] == 8, 'WORKRELATED'] = 'Yes (Sales and Related Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 9, 'WORKRELATED'] = 'Yes (Farming, Fishing, and Forestry Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 10, 'WORKRELATED'] = 'Yes (Installation, Maintenance, and Repair Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 11, 'WORKRELATED'] = 'Yes (Transportation and Material Moving Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 12, 'WORKRELATED'] = 'Yes (Management Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 13, 'WORKRELATED'] = 'Yes (Computer and Mathematical Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 14, 'WORKRELATED'] = 'Yes (Life, Physical, and Social Science Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 15, 'WORKRELATED'] = 'Yes (Legal Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 16, 'WORKRELATED'] = 'Yes (Arts, Design, Entertainment, Sports, and Media)'
data.loc[data['PATIENTSOCCUPATION'] == 17, 'WORKRELATED'] = 'Yes (Healthcare Support Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 18, 'WORKRELATED'] = 'Yes (Food Preparation and Serving Related)'
data.loc[data['PATIENTSOCCUPATION'] == 19, 'WORKRELATED'] = 'Yes (Personal Care and Service Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 20, 'WORKRELATED'] = 'Yes (Office and Administrative Support Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 21, 'WORKRELATED'] = 'Yes (Construction and Extraction Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 22, 'WORKRELATED'] = 'Yes (Production Occupations)'
data.loc[data['PATIENTSOCCUPATION'] == 23, 'WORKRELATED'] = 'Yes (Military Specific Occupations)'

data = data.drop(columns=['PATIENTSOCCUPATION'])

data['WORKRELATED'].value_counts(normalize=False, dropna=False)

No                                                          12799
Yes (Construction and Extraction Occupations)                 227
NaN                                                           168
Yes (Transportation and Material Moving Occupations)           49
Yes (Installation, Maintenance, and Repair Occupations)        48
Yes (Farming, Fishing, and Forestry Occupations)               22
Yes (Building and Grounds Cleaning and Maintenance)            22
Yes (Production Occupations)                                   14
Yes (Food Preparation and Serving Related)                     13
Yes (Sales and Related Occupations)                            11
Yes (Military Specific Occupations)                             6
Yes (Protective Service Occupations)                            6
Yes (Arts, Design, Entertainment, Sports, and Media)            5
Yes (Healthcare Practitioners and Technical Occupations)        5
Yes (Management Occupations)                                    3
Yes (Educa

In [32]:
#Change response values to strings for 'PREHOSPITALCARDIACARREST'.

data.loc[data['PREHOSPITALCARDIACARREST'] == 1, 'PREHOSPITALCARDIACARREST'] = 'Yes'
data.loc[data['PREHOSPITALCARDIACARREST'] == 2, 'PREHOSPITALCARDIACARREST'] = 'No'

data['PREHOSPITALCARDIACARREST'].value_counts(normalize=False, dropna=False)

No     13156
NaN      145
Yes      106
Name: PREHOSPITALCARDIACARREST, dtype: int64

In [33]:
#Change response values to strings for 'PRIMARYMETHODPAYMENT'.

data.loc[data['PRIMARYMETHODPAYMENT'] == 1, 'PRIMARYMETHODPAYMENT'] = 'Medicaid'
data.loc[data['PRIMARYMETHODPAYMENT'] == 2, 'PRIMARYMETHODPAYMENT'] = 'Not Billed (for any reason)'
data.loc[data['PRIMARYMETHODPAYMENT'] == 3, 'PRIMARYMETHODPAYMENT'] = 'Self-Pay'
data.loc[data['PRIMARYMETHODPAYMENT'] == 4, 'PRIMARYMETHODPAYMENT'] = 'Private/Commercial Insurance'
data.loc[data['PRIMARYMETHODPAYMENT'] == 6, 'PRIMARYMETHODPAYMENT'] = 'Medicare'
data.loc[data['PRIMARYMETHODPAYMENT'] == 7, 'PRIMARYMETHODPAYMENT'] = 'Other Government'
data.loc[data['PRIMARYMETHODPAYMENT'] == 10, 'PRIMARYMETHODPAYMENT'] = 'Other'

data['PRIMARYMETHODPAYMENT'].value_counts(normalize=False, dropna=False)

Private/Commercial Insurance    5462
Medicaid                        3998
Medicare                        1393
Self-Pay                        1350
Other Government                 548
NaN                              323
Other                            299
Not Billed (for any reason)       34
Name: PRIMARYMETHODPAYMENT, dtype: int64

In [34]:
#Merge protective device columns and drop former columns.

data.loc[data['PROTDEV_AIRBAG_PRESENT'] == 1, 'PROTDEV'] = 'Airbag Present'
data.loc[data['PROTDEV_CHILD_RESTRAINT'] == 1, 'PROTDEV'] = 'Child Restraint (booster seat or child car seat)'
data.loc[data['PROTDEV_EYE_PROTECT'] == 1, 'PROTDEV'] = 'Eye Protection'
data.loc[data['PROTDEV_HELMET'] == 1, 'PROTDEV'] = 'Helmet'
data.loc[data['PROTDEV_LAP_BELT'] == 1, 'PROTDEV'] = 'Lap Belt'
data.loc[data['PROTDEV_PER_FLOAT'] == 1, 'PROTDEV'] = 'Personal Floatation Device'
data.loc[data['PROTDEV_PROTECT_CLOTH'] == 1, 'PROTDEV'] = 'Protective Clothing'
data.loc[data['PROTDEV_PROTECT_GEAR'] == 1, 'PROTDEV'] = 'Protective Non-Clothing Gear'
data.loc[data['PROTDEV_SHOULDER_BELT'] == 1, 'PROTDEV'] = 'None'
data.loc[data['PROTDEV_OTHER'] == 1, 'PROTDEV'] = 'Shoulder Belt'
data.loc[data['PROTDEV_NONE'] == 1, 'PROTDEV'] = 'None'

data = data.drop(columns=['PROTDEV_AIRBAG_PRESENT', 'PROTDEV_CHILD_RESTRAINT', 'PROTDEV_EYE_PROTECT', 'PROTDEV_HELMET', 'PROTDEV_LAP_BELT', 'PROTDEV_PER_FLOAT', 'PROTDEV_PROTECT_CLOTH', 'PROTDEV_PROTECT_GEAR', 'PROTDEV_SHOULDER_BELT', 'PROTDEV_OTHER', 'PROTDEV_NONE'])

data['PROTDEV'].value_counts(normalize=False, dropna=False)

None                                                12073
NaN                                                   388
Airbag Present                                        324
Helmet                                                277
Lap Belt                                              198
Child Restraint (booster seat or child car seat)       70
Protective Clothing                                    30
Protective Non-Clothing Gear                           22
Shoulder Belt                                          20
Personal Floatation Device                              4
Eye Protection                                          1
Name: PROTDEV, dtype: int64

In [35]:
#Change response values to strings for 'RESPIRATORYASSISTANCE'.

data.loc[data['RESPIRATORYASSISTANCE'] == 1, 'RESPIRATORYASSISTANCE'] = 'Unassisted Respiratory Rate'
data.loc[data['RESPIRATORYASSISTANCE'] == 2, 'RESPIRATORYASSISTANCE'] = 'Assisted Respiratory Rate'

data['RESPIRATORYASSISTANCE'].value_counts(normalize=False, dropna=False)

Unassisted Respiratory Rate    11628
Assisted Respiratory Rate       1087
NaN                              692
Name: RESPIRATORYASSISTANCE, dtype: int64

In [36]:
#Change response values to strings for 'SEX'.

data.loc[data['SEX'] == 1, 'SEX'] = 'Male'
data.loc[data['SEX'] == 2, 'SEX'] = 'Female'
data.loc[data['SEX'] == 3, 'SEX'] = 'Non-Binary'

data['SEX'].value_counts(normalize=False, dropna=False)

Male          9389
Female        3952
NaN             64
Non-Binary       2
Name: SEX, dtype: int64

In [37]:
#Change response values to strings for 'SUPPLEMENTALOXYGEN'.

data.loc[data['SUPPLEMENTALOXYGEN'] == 1, 'SUPPLEMENTALOXYGEN'] = 'No Supplemental Oxygen'
data.loc[data['SUPPLEMENTALOXYGEN'] == 2, 'SUPPLEMENTALOXYGEN'] = 'Supplemental Oxygen'

data['SUPPLEMENTALOXYGEN'].value_counts(normalize=False, dropna=False)

No Supplemental Oxygen    10405
Supplemental Oxygen        2044
NaN                         958
Name: SUPPLEMENTALOXYGEN, dtype: int64

In [38]:
#Change response values to strings for 'TBIMIDLINESHIFT'.

data.loc[data['TBIMIDLINESHIFT'] == 1, 'TBIMIDLINESHIFT'] = 'Yes'
data.loc[data['TBIMIDLINESHIFT'] == 2, 'TBIMIDLINESHIFT'] = 'No'
data.loc[data['TBIMIDLINESHIFT'] == 3, 'TBIMIDLINESHIFT'] = 'Not Imaged (e.g. CT Scan, MRI)'

data['TBIMIDLINESHIFT'].value_counts(normalize=False, dropna=False)

No                                10583
Yes                                1550
NaN                                1101
Not Imaged (e.g. CT Scan, MRI)      173
Name: TBIMIDLINESHIFT, dtype: int64

In [39]:
#Change response values to strings for 'TBIPUPILLARYRESPONSE'.

data.loc[data['TBIPUPILLARYRESPONSE'] == 1, 'TBIPUPILLARYRESPONSE'] = 'Both reactive'
data.loc[data['TBIPUPILLARYRESPONSE'] == 2, 'TBIPUPILLARYRESPONSE'] = 'One reactive'
data.loc[data['TBIPUPILLARYRESPONSE'] == 3, 'TBIPUPILLARYRESPONSE'] = 'Neither reactive'

data['TBIPUPILLARYRESPONSE'].value_counts(normalize=False, dropna=False)

Both reactive       11370
NaN                  1499
Neither reactive      324
One reactive          214
Name: TBIPUPILLARYRESPONSE, dtype: int64

In [40]:
#Change response values to strings for 'TBIPUPILLARYRESPONSE'.

data.loc[data['TBIPUPILLARYRESPONSE'] == 1, 'TBIPUPILLARYRESPONSE'] = 'Both reactive'
data.loc[data['TBIPUPILLARYRESPONSE'] == 2, 'TBIPUPILLARYRESPONSE'] = 'One reactive'
data.loc[data['TBIPUPILLARYRESPONSE'] == 3, 'TBIPUPILLARYRESPONSE'] = 'Neither reactive'

data['TBIPUPILLARYRESPONSE'].value_counts(normalize=False, dropna=False)

Both reactive       11370
NaN                  1499
Neither reactive      324
One reactive          214
Name: TBIPUPILLARYRESPONSE, dtype: int64

In [41]:
#Change response values to strings for 'TRANSPORTMODE'.

data.loc[data['TRANSPORTMODE'] == 1, 'TRANSPORTMODE'] = 'Ground Ambulance'
data.loc[data['TRANSPORTMODE'] == 2, 'TRANSPORTMODE'] = 'Helicopter Ambulance'
data.loc[data['TRANSPORTMODE'] == 3, 'TRANSPORTMODE'] = 'Fixed-wing Ambulance'
data.loc[data['TRANSPORTMODE'] == 4, 'TRANSPORTMODE'] = 'Private/Public Vehicle/Walk-in'
data.loc[data['TRANSPORTMODE'] == 5, 'TRANSPORTMODE'] = 'Police'
data.loc[data['TRANSPORTMODE'] == 6, 'TRANSPORTMODE'] = 'Other'

data['TRANSPORTMODE'].value_counts(normalize=False, dropna=False)

Ground Ambulance                  8664
Private/Public Vehicle/Walk-in    2464
Helicopter Ambulance              2012
Fixed-wing Ambulance               109
Other                               79
NaN                                 51
Police                              28
Name: TRANSPORTMODE, dtype: int64

In [42]:
#Change response values to strings for 'TRAUMATYPE'.

data.loc[data['TRAUMATYPE'] == 1, 'TRAUMATYPE'] = 'Blunt'
data.loc[data['TRAUMATYPE'] == 2, 'TRAUMATYPE'] = 'Penetrating'
data.loc[data['TRAUMATYPE'] == 3, 'TRAUMATYPE'] = 'Burn'
data.loc[data['TRAUMATYPE'] == 4, 'TRAUMATYPE'] = 'Other/unspecified'
data.loc[data['TRAUMATYPE'] == 9, 'TRAUMATYPE'] = 'Hot Object/Substance'

data['TRAUMATYPE'].value_counts(normalize=False, dropna=False)

Blunt                12617
NaN                    484
Penetrating            177
Other/unspecified      128
Burn                     1
Name: TRAUMATYPE, dtype: int64

In [43]:
#Change response values to strings for 'VERIFICATIONLEVEL'.

data.loc[data['VERIFICATIONLEVEL'] == 1, 'VERIFICATIONLEVEL'] = 'Level I Trauma Center'
data.loc[data['VERIFICATIONLEVEL'] == 2, 'VERIFICATIONLEVEL'] = 'Level II Trauma Center'
data.loc[data['VERIFICATIONLEVEL'] == 3, 'VERIFICATIONLEVEL'] = 'Level III Trauma Center'

data['VERIFICATIONLEVEL'].value_counts(normalize=False, dropna=False)

NaN                        5406
Level I Trauma Center      5151
Level II Trauma Center     2401
Level III Trauma Center     449
Name: VERIFICATIONLEVEL, dtype: int64

In [44]:
print(list(pd.Series.to_frame(data['BEDSIZE'].value_counts(normalize=False, dropna=False)).index))


['201 to 400', 'More than 600', '401 to 600', '200 or fewer']


In [45]:
#Check data shape.

data.shape

(13407, 116)

#Exclusion Criteria

In [46]:
#Exclude pediatric patients.

before = data.shape[0]
data = data[(data['AGEYEARS'] >= 18)]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  13407
Number of patients after exclusion:  7163
Number of patients excluded with this criteria:  6244


In [47]:
#Exclude AIS injury severity score ≥ 3 to neck.

before = data.shape[0]
data = data[data['AISSEVERITY3'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  7163
Number of patients after exclusion:  7051
Number of patients excluded with this criteria:  112


In [48]:
#Exclude AIS injury severity score ≥ 3 to thorax.

before = data.shape[0]
data = data[data['AISSEVERITY4'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  7051
Number of patients after exclusion:  6233
Number of patients excluded with this criteria:  818


In [49]:
#Exclude AIS injury severity score ≥ 3 to abdomen.

before = data.shape[0]
data = data[data['AISSEVERITY5'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  6233
Number of patients after exclusion:  6175
Number of patients excluded with this criteria:  58


In [50]:
#Exclude AIS injury severity score ≥ 3 to spine.

before = data.shape[0]
data = data[data['AISSEVERITY6'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  6175
Number of patients after exclusion:  5693
Number of patients excluded with this criteria:  482


In [51]:
#Exclude AIS injury severity score ≥ 3 to upper extremity.

before = data.shape[0]
data = data[data['AISSEVERITY7'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  5693
Number of patients after exclusion:  5676
Number of patients excluded with this criteria:  17


In [52]:
#Exclude AIS injury severity score ≥ 3 to lower extremity.

before = data.shape[0]
data = data[data['AISSEVERITY8'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  5676
Number of patients after exclusion:  5552
Number of patients excluded with this criteria:  124


In [53]:
#Exclude AIS injury severity score ≥ 3 to unspecified body regions.

before = data.shape[0]
data = data[data['AISSEVERITY9'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  5552
Number of patients after exclusion:  5550
Number of patients excluded with this criteria:  2


In [54]:
#Exclude major polytrauma patients.

before = data.shape[0]
data = data[data['ISS'] < 27]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  5550
Number of patients after exclusion:  4790
Number of patients excluded with this criteria:  760


In [55]:
#Save data.

data.to_csv('/content/drive/MyDrive/NTDB-Epidural/clean_data.csv')

#Imputation

In [56]:
#Define numerical and categorical columns.

num_cols = list(data.select_dtypes('number').columns)
print('Numerical columns: {}'.format(num_cols), '\n')

cat_cols = list(data.select_dtypes('object').columns)
print('Categorical columns: {}'.format(cat_cols))

Numerical columns: ['AGEYEARS', 'WEIGHT', 'HEIGHT', 'SBP', 'PULSERATE', 'PULSEOXIMETRY', 'RESPIRATORYRATE', 'TEMPERATURE', 'GCSEYE', 'GCSVERBAL', 'GCSMOTOR', 'TOTALGCS', 'HOSPITALARRIVALDAYS', 'AISSEVERITY1', 'AISSEVERITY2', 'AISSEVERITY3', 'AISSEVERITY4', 'AISSEVERITY5', 'AISSEVERITY6', 'AISSEVERITY7', 'AISSEVERITY8', 'AISSEVERITY9', 'ISS', 'BLOOD4HOURS', 'PLASMA4HOURS', 'PLATELETS4HOURS', 'CRYOPRECIPITATE4HOURS', 'ALCOHOLSCREENRESULT', 'FINALDISCHARGEDAYS', 'TOTALICULOS'] 

Categorical columns: ['SEX', 'ETHNICITY', 'SUPPLEMENTALOXYGEN', 'RESPIRATORYASSISTANCE', 'PREHOSPITALCARDIACARREST', 'TBIPUPILLARYRESPONSE', 'TBIMIDLINESHIFT', 'CC_SMOKING', 'CC_ALCOHOLISM', 'CC_SUBSTANCEABUSE', 'CC_DIABETES', 'CC_HYPERTENSION', 'CC_CHF', 'CC_MI', 'CC_ANGINAPECTORIS', 'CC_CVA', 'CC_PAD', 'CC_COPD', 'CC_RENAL', 'CC_CIRRHOSIS', 'CC_BLEEDING', 'CC_DISCANCER', 'CC_CHEMO', 'CC_DEMENTIA', 'CC_ADHD', 'CC_MENTALPERSONALITY', 'CC_FUNCTIONAL', 'CC_PREGNANCY', 'CC_ANTICOAGULANT', 'CC_STEROID', 'CC_ADLC', 'TR

In [57]:
#Remove outcomes.

num_cols_remove = ['FINALDISCHARGEDAYS', 'TOTALICULOS']
cat_cols_remove = ['HC_CARDARREST', 'HC_CAUTI', 'HC_CLABSI', 'HC_CRBSI', 'HC_DEEPSSI', 'HC_DELIRIUM', 'HC_DRUGALCOHOLWITHDRAWAL', 'HC_DVTHROMBOSIS', 'HC_EMBOLISM', 'HC_EXTREMITYCS', 'HC_GRAFTFAIL', 'HC_INTUBATION', 'HC_KIDNEY', 'HC_MI', 'HC_ORGANSPACESSI', 'HC_OSTEOMYELITIS', 'HC_PNEUMONIA', 'HC_PRESSUREULCER', 'HC_RESPIRATORY', 'HC_RETURNOR', 'HC_SEPSIS', 'HC_STROKECVA', 'HC_SUPERFICIALSSI', 'HC_UNPLANNEDICU', 'HC_UTI', 'HC_VAPNEUMONIA', 'HOSPDISCHARGEDISPOSITION']

num_cols = [i for i in num_cols if i not in num_cols_remove]
cat_cols = [i for i in cat_cols if i not in cat_cols_remove]

In [58]:
#Check missing values for numerical columns.

missing_num = data[num_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

print(missing_num, '\n')

missing_num = pd.DataFrame(missing_num)

missing_num.columns = ['Value']

missing_num = missing_num[missing_num['Value'] > 0]

print('Numerical variables with missing values: ', list(missing_num.index), '\n')

print('Number of numerical variables with missing values: ', len(list(missing_num.index)), '\n')

missing_num = missing_num[missing_num['Value'] > 25]

missing_num = list(missing_num.index)

print('Excluded numerical variables: ', missing_num)

PLASMA4HOURS             39.10
PLATELETS4HOURS          39.10
CRYOPRECIPITATE4HOURS    39.10
HEIGHT                    8.75
TEMPERATURE               8.18
BLOOD4HOURS               8.10
GCSVERBAL                 4.38
GCSMOTOR                  4.38
GCSEYE                    4.36
WEIGHT                    4.26
TOTALGCS                  4.22
RESPIRATORYRATE           2.59
PULSEOXIMETRY             2.32
SBP                       2.28
PULSERATE                 1.96
HOSPITALARRIVALDAYS       0.77
ISS                       0.00
AGEYEARS                  0.00
AISSEVERITY8              0.00
AISSEVERITY9              0.00
AISSEVERITY2              0.00
AISSEVERITY7              0.00
AISSEVERITY6              0.00
AISSEVERITY5              0.00
AISSEVERITY4              0.00
AISSEVERITY3              0.00
AISSEVERITY1              0.00
ALCOHOLSCREENRESULT       0.00
dtype: float64 

Numerical variables with missing values:  ['PLASMA4HOURS', 'PLATELETS4HOURS', 'CRYOPRECIPITATE4HOURS', 'HEIGHT', 'T

In [59]:
#Drop numerical columns with missing values over 25%.

data.drop(missing_num, axis=1, inplace=True)

In [60]:
#Define new numerical columns.

num_cols = [x for x in num_cols if x not in missing_num]

In [61]:
#Impute missing numerical values.

num_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
data[num_cols] = num_imputer.fit_transform(data[num_cols])

In [62]:
#Check missing values for categorical columns.

missing_cat = data[cat_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

print(missing_cat, '\n')

missing_cat = pd.DataFrame(missing_cat)

missing_cat.columns = ['Value']

missing_cat = missing_cat[missing_cat['Value'] > 0]

print('Categorical variables with missing values: ', list(missing_cat.index), '\n')

print('Number of categorical variables with missing values: ', len(list(missing_cat.index)), '\n')

missing_cat = missing_cat[missing_cat['Value'] > 25]

missing_cat = list(missing_cat.index)

print('Excluded categorical variables: ', missing_cat)

ANTIBIOTICTHERAPY           95.07
VERIFICATIONLEVEL           25.14
TBIPUPILLARYRESPONSE        11.44
TBIMIDLINESHIFT              8.43
ICP                          8.08
SUPPLEMENTALOXYGEN           5.82
RESPIRATORYASSISTANCE        4.43
ETHNICITY                    4.34
PRIMARYMETHODPAYMENT         3.36
PROTDEV                      3.03
RACE                         2.44
WORKRELATED                  2.05
TRAUMATYPE                   1.38
MECHANISM                    1.04
INTENT                       0.94
PREHOSPITALCARDIACARREST     0.92
TRANSPORTMODE                0.61
ALCOHOLSCREEN                0.27
SEX                          0.27
HOSPITALTYPE                 0.08
CC_CIRRHOSIS                 0.00
DRGSCR_OPIOID                0.00
DRGSCR_BARBITURATE           0.00
DRGSCR_BENZODIAZEPINES       0.00
DRGSCR_CANNABINOID           0.00
DRGSCR_COCAINE               0.00
DRGSCR_ECSTASY               0.00
DRGSCR_METHADONE             0.00
DRGSCR_METHAMPHETAMINE       0.00
DRGSCR_OXYCODO

In [63]:
#Drop categorical columns with missing values over 25%.

data.drop(missing_cat, axis=1, inplace=True)

In [64]:
#Define new categorical columns.

cat_cols = [x for x in cat_cols if x not in missing_cat]

In [65]:
#Replace missing categorical values with 'Unknown'.

for col in cat_cols:
    data[col].fillna(value='Unknown', inplace=True)

#More Renaming and Merging Response Values

In [66]:
print(list(data['MECHANISM'].unique()))

['Struck By/Against', 'Fall', 'Other Specified, Not Elsewhere Classifiable', 'Transport, Other', 'MVT Motorcyclist', 'MVT Occupant', 'Other Specified and Classifiable', 'MVT Other', 'Cut/Pierce', 'Unspecified', 'Pedal Cyclist, Other', 'Natural/Environmental, Other', 'MVT Pedal Cyclist', 'MVT Pedestrian', 'Firearm', 'Unknown', 'MVT Unspecified', 'Pedestrian, Other', 'Machinery', 'Overexertion']


In [67]:
data.loc[data['SUPPLEMENTALOXYGEN'] == 'No Supplemental Oxygen', 'SUPPLEMENTALOXYGEN'] = 'No supplemental oxygen'
data.loc[data['SUPPLEMENTALOXYGEN'] == 'Supplemental Oxygen', 'SUPPLEMENTALOXYGEN'] = 'Supplemental oxygen'

print(list(pd.Series.to_frame(data['SUPPLEMENTALOXYGEN'].value_counts(normalize=False, dropna=False)).index))

['No supplemental oxygen', 'Supplemental oxygen', 'Unknown']


In [68]:
data.loc[data['RESPIRATORYASSISTANCE'] == 'Unassisted Respiratory Rate', 'RESPIRATORYASSISTANCE'] = 'Unassisted respiratory rate'
data.loc[data['RESPIRATORYASSISTANCE'] == 'Assisted Respiratory Rate', 'RESPIRATORYASSISTANCE'] = 'Assisted respiratory rate'

print(list(pd.Series.to_frame(data['RESPIRATORYASSISTANCE'].value_counts(normalize=False, dropna=False)).index))

['Unassisted respiratory rate', 'Assisted respiratory rate', 'Unknown']


In [69]:
data.loc[data['TBIMIDLINESHIFT'] == 'Not Imaged (e.g. CT Scan, MRI)', 'TBIMIDLINESHIFT'] = 'Not imaged/unknown'
data.loc[data['TBIMIDLINESHIFT'] == 'Unknown', 'TBIMIDLINESHIFT'] = 'Not imaged/unknown'

print(list(pd.Series.to_frame(data['TBIMIDLINESHIFT'].value_counts(normalize=False, dropna=False)).index))

['No', 'Yes', 'Not imaged/unknown']


In [70]:
data.loc[data['TRANSPORTMODE'] == 'Ground Ambulance', 'TRANSPORTMODE'] = 'Ground ambulance'
data.loc[data['TRANSPORTMODE'] == 'Private/Public Vehicle/Walk-in', 'TRANSPORTMODE'] = 'Private vehicle/public vehicle/walk-in'
data.loc[data['TRANSPORTMODE'] == 'Helicopter Ambulance', 'TRANSPORTMODE'] = 'Air ambulance'
data.loc[data['TRANSPORTMODE'] == 'Fixed-wing Ambulance', 'TRANSPORTMODE'] = 'Air ambulance'
data.loc[data['TRANSPORTMODE'] == 'Other', 'TRANSPORTMODE'] = 'Other/unknown'
data.loc[data['TRANSPORTMODE'] == 'Unknown', 'TRANSPORTMODE'] = 'Other/unknown'

print(list(pd.Series.to_frame(data['TRANSPORTMODE'].value_counts(normalize=False, dropna=False)).index))

['Ground ambulance', 'Private vehicle/public vehicle/walk-in', 'Air ambulance', 'Other/unknown', 'Police']


In [71]:
data.loc[data['TRAUMATYPE'] == 'Other/unspecified', 'TRAUMATYPE'] = 'Other/unknown'
data.loc[data['TRAUMATYPE'] == 'Unknown', 'TRAUMATYPE'] = 'Other/unknown'

print(list(pd.Series.to_frame(data['TRAUMATYPE'].value_counts(normalize=False, dropna=False)).index))

['Blunt', 'Other/unknown', 'Penetrating']


In [72]:
data.loc[data['INTENT'] == 'Other', 'INTENT'] = 'Other/undetermined/unknown'
data.loc[data['INTENT'] == 'Undetermined', 'INTENT'] = 'Other/undetermined/unknown'
data.loc[data['INTENT'] == 'Unknown', 'INTENT'] = 'Other/undetermined/unknown'

print(list(pd.Series.to_frame(data['INTENT'].value_counts(normalize=False, dropna=False)).index))

['Unintentional', 'Assault', 'Other/undetermined/unknown', 'Self-inflicted']


In [73]:
data.loc[data['MECHANISM'] == 'Other Specified, Not Elsewhere Classifiable', 'MECHANISM'] = 'Other/unspecified/unknown'
data.loc[data['MECHANISM'] == 'Other Specified and Classifiable', 'MECHANISM'] = 'Other/unspecified/unknown'
data.loc[data['MECHANISM'] == 'Unspecified', 'MECHANISM'] = 'Other/unspecified/unknown'
data.loc[data['MECHANISM'] == 'Unknown', 'MECHANISM'] = 'Other/unspecified/unknown'
data.loc[data['MECHANISM'] == 'Transport, Other', 'MECHANISM'] = 'Other transport'
data.loc[data['MECHANISM'] == 'MVT Other', 'MECHANISM'] = 'Other MVT'
data.loc[data['MECHANISM'] == 'MVT Unspecified', 'MECHANISM'] = 'Other MVT'
data.loc[data['MECHANISM'] == 'Pedal Cyclist, Other', 'MECHANISM'] = 'Other pedal cyclist'
data.loc[data['MECHANISM'] == 'Pedestrian, Other', 'MECHANISM'] = 'Other pedestrian'
data.loc[data['MECHANISM'] == 'Natural/Environmental, Other', 'MECHANISM'] = 'Natural/environmental'
data.loc[data['MECHANISM'] == 'Struck By/Against', 'MECHANISM'] = 'Struck by or against'
data.loc[data['MECHANISM'] == 'MVT Occupant', 'MECHANISM'] = 'MVT occupant'
data.loc[data['MECHANISM'] == 'MVT Motorcyclist', 'MECHANISM'] = 'MVT motorcyclist'
data.loc[data['MECHANISM'] == 'MVT Pedestrian', 'MECHANISM'] = 'MVT pedestrian'
data.loc[data['MECHANISM'] == 'MVT Pedal Cyclist', 'MECHANISM'] = 'MVT pedal cyclist'
data.loc[data['MECHANISM'] == 'Cut/Pierce', 'MECHANISM'] = 'Cut/pierce'

print(list(pd.Series.to_frame(data['MECHANISM'].value_counts(normalize=False, dropna=False)).index))

['Fall', 'Struck by or against', 'MVT occupant', 'MVT motorcyclist', 'Other/unspecified/unknown', 'Other transport', 'MVT pedestrian', 'Other MVT', 'Other pedal cyclist', 'MVT pedal cyclist', 'Firearm', 'Cut/pierce', 'Other pedestrian', 'Natural/environmental', 'Machinery', 'Overexertion']


In [74]:
data.loc[data['PROTDEV'] == 'Airbag Present', 'PROTDEV'] = 'Airbag present'
data.loc[data['PROTDEV'] == 'Lap Belt', 'PROTDEV'] = 'Lap belt'
data.loc[data['PROTDEV'] == 'Protective Clothing', 'PROTDEV'] = 'Protective clothing'
data.loc[data['PROTDEV'] == 'Protective Non-Clothing Gear', 'PROTDEV'] = 'Protective non-clothing gear'
data.loc[data['PROTDEV'] == 'Eye Protection', 'PROTDEV'] = 'Eye protection'

print(list(pd.Series.to_frame(data['PROTDEV'].value_counts(normalize=False, dropna=False)).index))

['None', 'Unknown', 'Airbag present', 'Helmet', 'Lap belt', 'Protective clothing', 'Protective non-clothing gear', 'Shoulder Belt', 'Eye protection']


In [75]:
data.loc[data['WORKRELATED'] == 'Unknown', 'WORKRELATED'] = 'Yes (Unknown)'

print(list(pd.Series.to_frame(data['WORKRELATED'].value_counts(normalize=False, dropna=False)).index))

['No', 'Yes (Construction and Extraction Occupations)', 'Yes (Unknown)', 'Yes (Transportation and Material Moving Occupations)', 'Yes (Installation, Maintenance, and Repair Occupations)', 'Yes (Farming, Fishing, and Forestry Occupations)', 'Yes (Building and Grounds Cleaning and Maintenance)', 'Yes (Food Preparation and Serving Related)', 'Yes (Production Occupations)', 'Yes (Sales and Related Occupations)', 'Yes (Arts, Design, Entertainment, Sports, and Media)', 'Yes (Military Specific Occupations)', 'Yes (Healthcare Practitioners and Technical Occupations)', 'Yes (Management Occupations)', 'Yes (Protective Service Occupations)', 'Yes (Education, Training, and Library Occupations)', 'Yes (Office and Administrative Support Occupations)', 'Yes (Computer and Mathematical Occupations)', 'Yes (Legal Occupations)', 'Yes (Personal Care and Service Occupations)']


In [76]:
data.loc[data['HOSPITALTYPE'] == 'For Profit', 'HOSPITALTYPE'] = 'For profit'

print(list(pd.Series.to_frame(data['HOSPITALTYPE'].value_counts(normalize=False, dropna=False)).index))

['Non-profit', 'For profit', 'Government', 'Unknown']


In [77]:
data.loc[data['PRIMARYMETHODPAYMENT'] == 'Private/Commercial Insurance', 'PRIMARYMETHODPAYMENT'] = 'Private/commercial insurance'
data.loc[data['PRIMARYMETHODPAYMENT'] == 'Self-Pay', 'PRIMARYMETHODPAYMENT'] = 'Self-pay'
data.loc[data['PRIMARYMETHODPAYMENT'] == 'Other Government', 'PRIMARYMETHODPAYMENT'] = 'Other government'
data.loc[data['PRIMARYMETHODPAYMENT'] == 'Not Billed (for any reason)', 'PRIMARYMETHODPAYMENT'] = 'Not billed'

print(list(pd.Series.to_frame(data['PRIMARYMETHODPAYMENT'].value_counts(normalize=False, dropna=False)).index))

['Private/commercial insurance', 'Medicaid', 'Medicare', 'Self-pay', 'Other government', 'Unknown', 'Other', 'Not billed']


In [78]:
print(list(pd.Series.to_frame(data['PRIMARYMETHODPAYMENT'].value_counts(normalize=False, dropna=False)).index))

['Private/commercial insurance', 'Medicaid', 'Medicare', 'Self-pay', 'Other government', 'Unknown', 'Other', 'Not billed']


In [79]:
#Change variable names to field names.

data_dictionary = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/Modified Data Dictionary (Selected).csv", encoding = 'latin1', index_col = None, low_memory = False)
FieldNames = dict(zip(data_dictionary['Variable'], data_dictionary['Field Name']))
data.columns = data.columns.map(FieldNames)

In [80]:
#Save imputed data.

data.to_csv('/content/drive/MyDrive/NTDB-Epidural/imputed_data.csv')

In [81]:
#Save data for Gradio.

gradio = data.copy()

gradio.columns = gradio.columns.str.replace(' ', '_', regex=True)
gradio.columns = gradio.columns.str.replace('-', '', regex=True)

gradio.to_csv('/content/drive/MyDrive/NTDB-Epidural/gradio_data.csv')

In [82]:
#Exclude outcomes from scaling.

num_cols = list(data.select_dtypes('number').columns)
num_cols_remove = ['Total Length of Stay', 'Total ICU Length of Stay']
num_cols = [i for i in num_cols if i not in num_cols_remove]

In [83]:
#Normalize data.

data[num_cols] = MinMaxScaler().fit_transform(data[num_cols])

In [84]:
#Save scaled data.

data.to_csv('/content/drive/MyDrive/NTDB-Epidural/scaled_data.csv')

In [85]:
#Exclude outcomes from one hot encoding.

cat_cols = list(data.select_dtypes('object').columns)
cat_cols_remove = ['Hospital Complication - Cardiac Arrest with CPR', 'Hospital Complication - Catheter-Associated Urinary Tract Infection (CAUTI)', 'Hospital Complication - Central Line-Associated Bloodstream Infection (CLABSI)', 'Hospital Complication - Catheter-Related Blood Stream Infection', 'Hospital Complication - Deep Surgical Site Infection', 'Hospital Complication - Delirium', 'Hospital Complication - Alcohol Withdrawal Syndrome', 'Hospital Complication - Deep Vein Thrombosis (DVT)', 'Hospital Complication - Pulmonary Embolism', 'Hospital Complication - Extremity Compartment Syndrome', 'Hospital Complication - Graft or Prosthesis or Flap Failure', 'Hospital Complication - Unplanned Intubation', 'Hospital Complication - Acute Kidney Injury', 'Hospital Complication - Myocardial Infarction', 'Hospital Complication - Organ or Space Surgical Site Infection', 'Hospital Complication - Osteomyelitis', 'Hospital Complication - Pneumonia', 'Hospital Complication - Pressure or Decubitis Ulcer', 'Hospital Complication - Acute Respiratory Distress Syndrome (ARDS)', 'Hospital Complication - Unplanned Return to the OR', 'Hospital Complication - Severe Sepsis', 'Hospital Complication - Stroke  or  CVA', 'Hospital Complication - Superficial Surgical Site Infection', 'Hospital Complication - Unplanned Admission to the ICU', 'Hospital Complication - Urinary Tract Infection', 'Hospital Complication - Ventilator-Associated Pneumonia (VAP)', 'Discharge Disposition']
cat_cols = [i for i in cat_cols if i not in cat_cols_remove]

In [86]:
#One hot encoding for categorical values.

data = pd.get_dummies(data, columns = cat_cols, drop_first = False, prefix_sep=': ', )

In [87]:
#Save final data.

data.to_csv('/content/drive/MyDrive/NTDB-Epidural/final_data.csv')